In [ ]:
from skimage import io
import json
my_img = io.imread(r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\images\ffba62c94141.jpg')
io.imshow(my_img)
io.show()
j = r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\annotations\ffba62c94141.json'
f = open(j)
data = json.load(f)
f.close()

In [ ]:
data

In [ ]:
path = r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\images'
json_path = r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\annotations'
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.nn import functional
import json
import pandas as pd

bsize=256

img_names = os.listdir(path)
#json_names = os.listdir(json_path)
i=0
#ADD TRAIN TEST SPLIT HERE TO IMG_NAMES
from sklearn.model_selection import train_test_split
train, test = train_test_split(img_names, test_size=0.1, random_state=5)

images = np.empty(shape = (bsize, 3, 256, 256))
labels = np.chararray(bsize, itemsize=20)
for idx, name in enumerate(img_names):
    if (i==bsize):
        break
    img_name = path + '\\' + name
    json_name = json_path + '\\' + name
    json_name = json_name.replace(json_name[len(json_name) - 3:], 'json')
    # Use you favourite library to load the image
    image = plt.imread(img_name)
    image = image.copy()
    image = torch.from_numpy(image)
    image = torch.unsqueeze(image, dim=0)
    image = torch.movedim(image, source=3, destination=1)
    image = functional.interpolate(image, size = (256,256))
    
    f = open(json_name)
    data = json.load(f)
    t = data['chart-type']
    images[idx] = image
    labels[idx] = t
    f.close()
    
    #image_resized = functional.interpolate(image, size = (128,128))
    i+=1

In [ ]:
images.shape

In [ ]:
#my_img=images[45]
io.imshow(my_img)
io.show()

In [ ]:
#labels

In [ ]:
efnet_weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
efnet_model = torchvision.models.efficientnet_b0(weights = efnet_weights)

In [ ]:
## Loop through each parameter and set `requires_grad` to false
for param in efnet_model.features.parameters():
    param.requires_grad = False

In [ ]:
## Replace the "classifier" layer with one for our application
efnet_model.classifier = torch.nn.Sequential( 
    torch.nn.Linear(in_features=1280, out_features=4, bias=True))

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import OneHotEncoder
from pandas import get_dummies

train_y=labels
train_X=images

encode_list = [b'vertical_bar',b'scatter',b'dot',b'line']
# Create an instance of the OneHotEncoder class and fit it on the data
encoder = OneHotEncoder(sparse=False)
encoder.fit(train_y.reshape(-1, 1))

# Transform the data using the fitted encoder
encoded_data = encoder.transform(train_y.reshape(-1, 1))

# Map the encoded data to the desired format
train_y = [encoder.categories_[0].tolist().index(category) for category in encoder.inverse_transform(encoded_data).flatten()]

print(train_y)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

y_tensor = torch.tensor(train_y)

### Convert to numpy array then reshape to 900 by 28 by 28
mnist_unflattened = train_X
mnist_unflattened = mnist_unflattened.reshape(256, 3, 256, 256)

## Convert to tensor
mnist_tensor = torch.from_numpy(mnist_unflattened)
mnist_tensor =  torch.unsqueeze(mnist_tensor, dim=1)

## Transform to proper input shape
new_mnist_tensors = mnist_tensor.expand(-1, -1, -1, -1, -1)

train_loader = DataLoader(TensorDataset(new_mnist_tensors.type(torch.FloatTensor), y_tensor.type(torch.LongTensor)), batch_size=32)

In [ ]:
## Hyperparms
epochs = 100
lrate = 0.1

## Cost Function
from torch import nn
cost_fn = nn.NLLLoss()

## Network model
torch.manual_seed(7) # For reproduction purposes (should be minor since only the last layers are randomly intialized)
net = efnet_model

## Optimizer (using ADAM, a more flexible algorithm than SGD this time)
optimizer = torch.optim.Adam(net.parameters(), lr=lrate)

## Initial values for cost tracking
track_cost = np.zeros(epochs)
cur_cost = 0.0

## Loop through the data
for epoch in range(epochs):
    
    cur_cost = 0.0
    correct = 0.0
    
    ## train_loader is iterable and numbers knows the batch
    for i, data in enumerate(train_loader, 0):
        
        ## The input tensor and labels tensor for the current batch
        inputs, labels = data
        ## Clear the gradient from the previous batch
        optimizer.zero_grad()
        
        ## Provide the input tensor into the network to get outputs
        inputs = inputs.reshape((32, 3, 256, 256))
        outputs = net(inputs)
        
        ## Calculate the cost for the current batch
        cost = cost_fn(outputs.log_softmax(dim=1), labels)
        
        ## Calculate the gradient
        cost.backward()
        
        ## Update the model parameters using the gradient
        optimizer.step()
        
        ## Track the current cost (accumulating across batches)
        cur_cost += cost.item()
    
    ## Store the accumulated cost at each epoch
    track_cost[epoch] = cur_cost
    print(f"Epoch: {epoch} Cost: {cur_cost}") ## Uncomment this if you want printed updates


In [ ]:
#Take 2

In [1]:
path = r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\images'
json_path = r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\annotations'
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.nn import functional
import json
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import OneHotEncoder
from pandas import get_dummies

bsize=256

img_names = os.listdir(path)
#img_name = path + '\\' + name
#json_names = os.listdir(json_path)
i=0
#ADD TRAIN TEST SPLIT HERE TO IMG_NAMES
from sklearn.model_selection import train_test_split
train, test = train_test_split(img_names, test_size=0.99, random_state=5)

images = np.empty(shape = (bsize, 3, 256, 256))
labels = np.chararray(bsize, itemsize=20)

In [2]:
train_y = train.copy()
train_X = train.copy()
for i in range(len(train)):
    train_X[i] = path + '\\' + img_names[i]
    train_y[i] = json_path + '\\' + img_names[i]
    train_y[i] = train_y[i].replace(train_y[i][len(train_y[i]) - 3:], 'json')

In [3]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

In [4]:
class MyDataset(Dataset):
    def __init__(self, image_paths, label_paths, transform=None):
        self.image_paths = image_paths
        self.label_paths = label_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image = Image.open(self.image_paths[index])
        f = open(self.label_paths[index])
        data = json.load(f)
        t = data['chart-type']
        #print(t)
        if (t=='scatter'):
            label = 0
        elif (t=='vertical_bar'):
            label = 1
        elif (t=='dot'):
            label = 2
        else:
            label = 3
        f.close()
        
        if self.transform:
            image = self.transform(image)
            #label = self.transform(label)
            
        return image, label

# Define the transform to resize and normalize the images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create a dataset from the image and label file paths
dataset = MyDataset(train_X, train_y, transform=transform)

# Create a dataloader to load the data in batches
batch_size = 256
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [5]:
efnet_weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
efnet_model = torchvision.models.efficientnet_b0(weights = efnet_weights)
## Loop through each parameter and set `requires_grad` to false
for param in efnet_model.features.parameters():
    param.requires_grad = False

## Replace the "classifier" layer with one for our application
efnet_model.classifier = torch.nn.Sequential( 
    torch.nn.Linear(in_features=1280, out_features=4, bias=True))

In [6]:
#encoder = OneHotEncoder(sparse=False)

# Train the neural network on the data using batches
from torch import nn
epochs = 10
lrate = 0.1
cost_fn = nn.NLLLoss()
net = efnet_model
optimizer = torch.optim.Adam(net.parameters(), lr=lrate)
track_cost = np.zeros(epochs)

for epoch in range(epochs):
    cur_cost = 0.0
    correct = 0.0
    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        inputs = inputs.reshape(inputs.size())
        outputs = net(inputs)
        #print(labels)
        
        cost = cost_fn(outputs.log_softmax(dim=1), labels)
        cost.backward()
        optimizer.step()
        cur_cost += cost.item()
        
    track_cost[epoch] = cur_cost
    print(f"Epoch: {epoch} Cost: {cur_cost}")

Epoch: 0 Cost: 12.282479166984558
Epoch: 1 Cost: 2.9375903010368347
Epoch: 2 Cost: 2.6073168516159058
Epoch: 3 Cost: 0.5780415870249271
Epoch: 4 Cost: 0.7651243954896927
Epoch: 5 Cost: 0.6312493895529769
Epoch: 6 Cost: 0.715614378452301
Epoch: 7 Cost: 0.4981958493590355
Epoch: 8 Cost: 0.23268637154251337
Epoch: 9 Cost: 0.1204118556343019


In [7]:
#ADD TEST CODE TO MAKE SURE IT WORKS
test_y = test.copy()
test_X = test.copy()
i=0
for i in range(len(test)):
    test_X[i] = path + '\\' + img_names[i]
    test_y[i] = json_path + '\\' + img_names[i]
    test_y[i] = test_y[i].replace(test_y[i][len(test_y[i]) - 3:], 'json')

In [8]:
dataset = MyDataset(test_X, test_y, transform=transform)

# Create a dataloader to load the data in batches
batch_size = 256
test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [9]:
# set model to evaluation mode
#net.eval()

# iterate through test data and generate predictions
#with torch.no_grad():
    #predicted = []
    #for data in test_loader:
        #inputs, _ = data
        #inputs = inputs.reshape(inputs.size())
        #outputs = net(inputs)
        #_, pred = torch.max(outputs.data, 1)
        #predicted.append(pred.numpy())

# convert predicted labels to numpy array
#predicted = np.concatenate(predicted)


In [10]:
#predicted

In [13]:
# set model to evaluation mode
net.eval()

# iterate through test data and generate predictions
with torch.no_grad():
    predicted = []
    actual = []
    for data in test_loader:
        inputs, labels = data
        #inputs = inputs.reshape(inputs.size())
        outputs = net(inputs)
        _, pred = torch.max(outputs.data, 1)
        predicted.append(pred.numpy())
        actual.append(labels.numpy())
        print(pred)
        print(labels)
        if (pred != labels):
            print(inputs)

# concatenate predicted and actual labels
predicted = np.concatenate(predicted)
actual = np.concatenate(actual)

# compute accuracy
accuracy = np.mean(predicted == actual)
print(accuracy)


tensor([1, 1, 2, 3, 3, 0, 3, 1, 3, 3, 3, 2, 1, 1, 1, 1, 0, 1, 1, 2, 0, 2, 1, 3,
        3, 2, 0, 3, 3, 2, 1, 2, 0, 0, 2, 1, 1, 1, 1, 3, 3, 1, 1, 0, 1, 1, 3, 0,
        1, 1, 0, 3, 1, 1, 3, 0, 1, 3, 2, 3, 1, 1, 3, 0, 3, 1, 1, 3, 0, 1, 1, 3,
        1, 3, 1, 3, 1, 1, 1, 2, 2, 3, 0, 3, 3, 3, 3, 1, 1, 3, 3, 1, 3, 1, 3, 3,
        3, 1, 1, 1, 3, 1, 2, 1, 1, 1, 0, 2, 3, 3, 3, 0, 3, 1, 1, 3, 1, 0, 1, 1,
        3, 1, 0, 1, 0, 0, 0, 0, 3, 3, 1, 0, 1, 1, 3, 0, 1, 0, 1, 1, 3, 2, 3, 3,
        3, 0, 2, 3, 3, 3, 3, 3, 3, 0, 0, 1, 0, 1, 3, 3, 1, 1, 1, 3, 3, 3, 3, 1,
        3, 3, 3, 3, 3, 0, 1, 1, 1, 3, 1, 0, 3, 3, 1, 3, 3, 3, 3, 0, 3, 3, 0, 2,
        1, 3, 3, 1, 0, 0, 1, 1, 1, 3, 0, 3, 3, 3, 1, 3, 1, 0, 0, 0, 1, 3, 1, 3,
        0, 1, 3, 0, 3, 1, 3, 2, 2, 0, 3, 0, 3, 2, 1, 2, 3, 1, 3, 3, 0, 0, 1, 0,
        0, 1, 2, 3, 3, 1, 1, 3, 3, 2, 1, 2, 3, 3, 2, 0])
tensor([1, 1, 2, 3, 3, 0, 3, 1, 3, 3, 3, 2, 1, 1, 1, 1, 0, 1, 1, 2, 0, 2, 1, 3,
        3, 2, 0, 3, 3, 2, 1, 2, 0, 0, 2, 1, 1, 1, 1, 3, 3, 1, 1

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [14]:
j = json_path + '\\' + '18a1ca02e41e.json'

In [15]:
j = r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\annotations\ffba62c94141.json'
f = open(j)
data = json.load(f)

In [16]:
data

{'source': 'generated',
 'chart-type': 'scatter',
 'plot-bb': {'height': 182, 'width': 413, 'x0': 72, 'y0': 64},
 'text': [{'id': 0,
   'polygon': {'x0': 58,
    'x1': 442,
    'x2': 442,
    'x3': 58,
    'y0': 6,
    'y1': 6,
    'y2': 47,
    'y3': 47},
   'text': 'TOTAL ARMY PERSONAL IN MIDDLE EAST & NORTH AFRICA (EXCLUDING HIGH INCOME)',
   'role': 'chart_title'},
  {'id': 1,
   'polygon': {'x0': 7,
    'x1': 20,
    'x2': 20,
    'x3': 7,
    'y0': 72,
    'y1': 72,
    'y2': 163,
    'y3': 163},
   'text': 'Soldiers (*10000)',
   'role': 'axis_title'},
  {'id': 2,
   'polygon': {'x0': 264,
    'x1': 289,
    'x2': 289,
    'x3': 264,
    'y0': 274,
    'y1': 274,
    'y2': 285,
    'y3': 285},
   'text': 'Year',
   'role': 'axis_title'},
  {'id': 3,
   'polygon': {'x0': 26,
    'x1': 63,
    'x2': 63,
    'x3': 26,
    'y0': 58,
    'y1': 58,
    'y2': 69,
    'y3': 69},
   'text': '400.00',
   'role': 'tick_label'},
  {'id': 4,
   'polygon': {'x0': 27,
    'x1': 63,
    'x2': 6

In [17]:
data['data-series']

[{'x': 1985.0567260940034, 'y': 303.93845026561644},
 {'x': 1991.92868719611, 'y': 242.38871588202977},
 {'x': 1994.0032414910859, 'y': 237.99230628320217},
 {'x': 1996.0129659643435, 'y': 334.7133174574098},
 {'x': 1997.9578606158834, 'y': 339.10972705623743},
 {'x': 2000.9400324149108, 'y': 324.45502839347876},
 {'x': 2003.9222042139384, 'y': 287.81828173658187},
 {'x': 2006.904376012966, 'y': 327.3859681260305},
 {'x': 2009.9513776337114, 'y': 361.09177505037553},
 {'x': 2011.8962722852511, 'y': 340.5751969225133},
 {'x': 2013.905996758509, 'y': 271.6981132075473},
 {'x': 2015.9805510534845, 'y': 276.09452280637487},
 {'x': 2018.0551053484603, 'y': 290.7492214691336}]

In [18]:
f.close()

In [39]:
j = r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\annotations\ffbe953e00a1.json'
f = open(j)
data = json.load(f)

In [40]:
data

{'source': 'generated',
 'chart-type': 'vertical_bar',
 'plot-bb': {'height': 180, 'width': 426, 'x0': 63, 'y0': 27},
 'text': [{'id': 0,
   'polygon': {'x0': 100,
    'x1': 402,
    'x2': 402,
    'x3': 100,
    'y0': 4,
    'y1': 4,
    'y2': 19,
    'y3': 19},
   'text': 'CO2 Emisions by burning coal for the year 2016',
   'role': 'chart_title'},
  {'id': 1,
   'polygon': {'x0': 253,
    'x1': 298,
    'x2': 298,
    'x3': 253,
    'y0': 265,
    'y1': 265,
    'y2': 277,
    'y3': 277},
   'text': 'Country',
   'role': 'axis_title'},
  {'id': 2,
   'polygon': {'x0': 3,
    'x1': 16,
    'x2': 16,
    'x3': 3,
    'y0': 39,
    'y1': 39,
    'y2': 195,
    'y3': 195},
   'text': 'Total CO2 Emissions (*10000)',
   'role': 'axis_title'},
  {'id': 3,
   'polygon': {'x0': 18,
    'x1': 55,
    'x2': 55,
    'x3': 18,
    'y0': 21,
    'y1': 21,
    'y2': 33,
    'y3': 33},
   'text': '160000',
   'role': 'tick_label'},
  {'id': 4,
   'polygon': {'x0': 18,
    'x1': 55,
    'x2': 55,
   

In [41]:
f.close()

In [42]:
data['data-series']

[{'x': 'Sri Lanka', 'y': 35785.58934955088},
 {'x': 'Sweden', 'y': 91386.3255454662},
 {'x': 'Switzerland', 'y': 48207.03041459579},
 {'x': 'Syria', 'y': 28096.125833094513},
 {'x': 'Taiwan', 'y': 115637.71048198247},
 {'x': 'Tajikistan', 'y': 88428.83957759837},
 {'x': 'Tanzania', 'y': 68909.43218967065},
 {'x': 'Thailand', 'y': 106173.75538480538},
 {'x': 'Uzbekistan', 'y': 99075.78906192258},
 {'x': 'Venezuela', 'y': 52939.00796318434},
 {'x': 'Vietnam', 'y': 128059.15154702736},
 {'x': 'World', 'y': 145212.5701606608},
 {'x': 'Yemen', 'y': 98484.291868349},
 {'x': 'Zambia', 'y': 78373.38728684772},
 {'x': 'Zimbabwe', 'y': 98484.291868349}]

In [43]:
len(data['data-series'])

15

In [44]:
data['data-series']

[{'x': 'Sri Lanka', 'y': 35785.58934955088},
 {'x': 'Sweden', 'y': 91386.3255454662},
 {'x': 'Switzerland', 'y': 48207.03041459579},
 {'x': 'Syria', 'y': 28096.125833094513},
 {'x': 'Taiwan', 'y': 115637.71048198247},
 {'x': 'Tajikistan', 'y': 88428.83957759837},
 {'x': 'Tanzania', 'y': 68909.43218967065},
 {'x': 'Thailand', 'y': 106173.75538480538},
 {'x': 'Uzbekistan', 'y': 99075.78906192258},
 {'x': 'Venezuela', 'y': 52939.00796318434},
 {'x': 'Vietnam', 'y': 128059.15154702736},
 {'x': 'World', 'y': 145212.5701606608},
 {'x': 'Yemen', 'y': 98484.291868349},
 {'x': 'Zambia', 'y': 78373.38728684772},
 {'x': 'Zimbabwe', 'y': 98484.291868349}]

In [45]:
x = "x,"
for i in range(len(data['data-series'])):
    x += str(data['data-series'][i]['x'])
    x += ";"
x=x[:-1] + ', ' + data['chart-type']

In [46]:
x

'x,Sri Lanka;Sweden;Switzerland;Syria;Taiwan;Tajikistan;Tanzania;Thailand;Uzbekistan;Venezuela;Vietnam;World;Yemen;Zambia;Zimbabwe, vertical_bar'

In [47]:
y = "y,"
for i in range(len(data['data-series'])):
    y += str(data['data-series'][i]['y'])
    y += ";"
y=y[:-1] + ', ' + data['chart-type']

In [48]:
y

'y,35785.58934955088;91386.3255454662;48207.03041459579;28096.125833094513;115637.71048198247;88428.83957759837;68909.43218967065;106173.75538480538;99075.78906192258;52939.00796318434;128059.15154702736;145212.5701606608;98484.291868349;78373.38728684772;98484.291868349, vertical_bar'

In [49]:
output = x + "\n" + y
output

'x,Sri Lanka;Sweden;Switzerland;Syria;Taiwan;Tajikistan;Tanzania;Thailand;Uzbekistan;Venezuela;Vietnam;World;Yemen;Zambia;Zimbabwe, vertical_bar\ny,35785.58934955088;91386.3255454662;48207.03041459579;28096.125833094513;115637.71048198247;88428.83957759837;68909.43218967065;106173.75538480538;99075.78906192258;52939.00796318434;128059.15154702736;145212.5701606608;98484.291868349;78373.38728684772;98484.291868349, vertical_bar'

In [51]:
data['text'][3]['text']

'160000'

In [52]:
len(data['text'])

27

In [74]:
data['text']

[{'id': 0,
  'polygon': {'x0': 100,
   'x1': 402,
   'x2': 402,
   'x3': 100,
   'y0': 4,
   'y1': 4,
   'y2': 19,
   'y3': 19},
  'text': 'CO2 Emisions by burning coal for the year 2016',
  'role': 'chart_title'},
 {'id': 1,
  'polygon': {'x0': 253,
   'x1': 298,
   'x2': 298,
   'x3': 253,
   'y0': 265,
   'y1': 265,
   'y2': 277,
   'y3': 277},
  'text': 'Country',
  'role': 'axis_title'},
 {'id': 2,
  'polygon': {'x0': 3,
   'x1': 16,
   'x2': 16,
   'x3': 3,
   'y0': 39,
   'y1': 39,
   'y2': 195,
   'y3': 195},
  'text': 'Total CO2 Emissions (*10000)',
  'role': 'axis_title'},
 {'id': 3,
  'polygon': {'x0': 18,
   'x1': 55,
   'x2': 55,
   'x3': 18,
   'y0': 21,
   'y1': 21,
   'y2': 33,
   'y3': 33},
  'text': '160000',
  'role': 'tick_label'},
 {'id': 4,
  'polygon': {'x0': 18,
   'x1': 55,
   'x2': 55,
   'x3': 18,
   'y0': 43,
   'y1': 43,
   'y2': 55,
   'y3': 55},
  'text': '140000',
  'role': 'tick_label'},
 {'id': 5,
  'polygon': {'x0': 19,
   'x1': 55,
   'x2': 55,
   'x

In [70]:
ylab = ""
for i in range(3, int((len(data['text'])-3)/2)):
    ylab = ylab + data['text'][i]['text'] + ";"

In [71]:
ylab

'160000;140000;120000;100000;80000;60000;40000;20000;0;'

In [72]:
xlab = ""
for i in range(int((len(data['text'])-3)/2), len(data['text'])):
    xlab = xlab + data['text'][i]['text'] + ";"

In [73]:
xlab

'Sri Lanka;Sweden;Switzerland;Syria;Taiwan;Tajikistan;Tanzania;Thailand;Uzbekistan;Venezuela;Vietnam;World;Yemen;Zambia;Zimbabwe;'

In [ ]:
efnet_weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
efnet_model = torchvision.models.efficientnet_b0(weights = efnet_weights)

In [ ]:
efnet_model

In [ ]:
#encoder = OneHotEncoder(sparse=False)

# Train the neural network on the data using batches
from torch import nn
epochs = 100
lrate = 0.1
cost_fn = nn.NLLLoss()
net = efnet_model
optimizer = torch.optim.Adam(net.parameters(), lr=lrate)
track_cost = np.zeros(epochs)

for epoch in range(epochs):
    cur_cost = 0.0
    correct = 0.0
    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        inputs = inputs.reshape(inputs.size())
        outputs = net(inputs)
        #print(labels)
        
        cost = cost_fn(outputs.log_softmax(dim=1), labels)
        cost.backward()
        optimizer.step()
        cur_cost += cost.item()
        
    track_cost[epoch] = cur_cost
    print(f"Epoch: {epoch} Cost: {cur_cost}")

In [ ]:
#take 3

In [ ]:
path = r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\images'
json_path = r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\annotations'
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.nn import functional
import json
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import OneHotEncoder
from pandas import get_dummies

bsize=256

img_names = os.listdir(path)
#img_name = path + '\\' + name
#json_names = os.listdir(json_path)
i=0
#ADD TRAIN TEST SPLIT HERE TO IMG_NAMES
from sklearn.model_selection import train_test_split
train, test = train_test_split(img_names, test_size=0.9, random_state=5)

images = np.empty(shape = (bsize, 3, 256, 256))
labels = np.chararray(bsize, itemsize=20)

In [ ]:
train_y = train.copy()
train_X = train.copy()
for i in range(len(train)):
    train_X[i] = path + '\\' + img_names[i]
    train_y[i] = json_path + '\\' + img_names[i]
    train_y[i] = train_y[i].replace(train_y[i][len(train_y[i]) - 3:], 'json')

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

In [ ]:
class MyDataset(Dataset):
    def __init__(self, image_paths, label_paths, transform=None):
        self.image_paths = image_paths
        self.label_paths = label_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image = Image.open(self.image_paths[index])
        f = open(self.label_paths[index])
        data = json.load(f)
        t = data['chart-type']
        #x
        x = "x,"
        for i in range(len(data['data-series'])):
            x += str(data['data-series'][i]['x'])
            x += ";"
        x=x[:-1] + ', ' + data['chart-type']
        #y
        y = "y,"
        for i in range(len(data['data-series'])):
            y += str(data['data-series'][i]['y'])
            y += ";"
        y=y[:-1] + ', ' + data['chart-type']
        label = x + "\n" + y
        #print(label)
        f.close()
        
        if self.transform:
            image = self.transform(image)
            #label = self.transform(label)
            
        return image, label

# Define the transform to resize and normalize the images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create a dataset from the image and label file paths
dataset = MyDataset(train_X, train_y, transform=transform)

# Create a dataloader to load the data in batches
batch_size = 256
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
efnet_weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
efnet_model = torchvision.models.efficientnet_b0(weights = efnet_weights)
## Loop through each parameter and set `requires_grad` to false
for param in efnet_model.features.parameters():
    param.requires_grad = False

## Replace the "classifier" layer with one for our application
efnet_model.classifier = torch.nn.Sequential( 
    torch.nn.Linear(in_features=1280, out_features=4, bias=True))

In [ ]:
#encoder = OneHotEncoder(sparse=False)

# Train the neural network on the data using batches
from torch import nn
epochs = 100
lrate = 0.1
cost_fn = nn.NLLLoss()
net = efnet_model
optimizer = torch.optim.Adam(net.parameters(), lr=lrate)
track_cost = np.zeros(epochs)

for epoch in range(epochs):
    cur_cost = 0.0
    correct = 0.0
    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        #inputs = inputs.reshape(inputs.size())
        outputs = net(inputs)
        #print(labels)
        labels = torch.tensor(labels)
        
        cost = cost_fn(outputs.log_softmax(dim=1), labels)
        cost.backward()
        optimizer.step()
        cur_cost += cost.item()
        
    track_cost[epoch] = cur_cost
    print(f"Epoch: {epoch} Cost: {cur_cost}")

In [ ]:
#ADD TEST CODE TO MAKE SURE IT WORKS
test_y = test.copy()
test_X = test.copy()
i=0
for i in range(len(test)):
    test_X[i] = path + '\\' + img_names[i]
    test_y[i] = json_path + '\\' + img_names[i]
    test_y[i] = test_y[i].replace(test_y[i][len(test_y[i]) - 3:], 'json')

In [ ]:
dataset = MyDataset(test_X, test_y, transform=transform)

# Create a dataloader to load the data in batches
batch_size = 256
test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
#TODO
#LOOK AT HOW WELL THIS MODEL PERFORMS BY FIGURING OUT KAGGLE SCORING

In [ ]:
#take 4

In [ ]:
path = r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\images'
json_path = r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\annotations'
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.nn import functional
import json
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import OneHotEncoder
from pandas import get_dummies

bsize=256

img_names = os.listdir(path)
#img_name = path + '\\' + name
#json_names = os.listdir(json_path)
i=0
#ADD TRAIN TEST SPLIT HERE TO IMG_NAMES
from sklearn.model_selection import train_test_split
train, test = train_test_split(img_names, test_size=0.9, random_state=5)

images = np.empty(shape = (bsize, 3, 256, 256))
labels = np.chararray(bsize, itemsize=20)

In [ ]:
train_y = train.copy()
train_X = train.copy()
for i in range(len(train)):
    train_X[i] = path + '\\' + img_names[i]
    train_y[i] = json_path + '\\' + img_names[i]
    train_y[i] = train_y[i].replace(train_y[i][len(train_y[i]) - 3:], 'json')

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from torch import nn

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

# define a custom dataset class for the new input format
class GraphDataset(Dataset):
    def __init__(self, data):
        self.x = torch.tensor([list(map(float, data['x'].split(';'))) for i in range(len(data))])
        self.y = torch.tensor([list(map(float, data['y'].split(';'))) for i in range(len(data))])
        self.graph_type = torch.tensor([self._map_graph_type(t) for t in data['type']])
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return (torch.cat((self.x[idx], self.y[idx], self.graph_type[idx]), dim=0),)
    
    def _map_graph_type(self, t):
        # define a dictionary mapping each graph type to a numerical value
        graph_type_map = {'scatter': 0, 'line': 1, 'bar': 2, 'dot': 3}
        return graph_type_map[t]

# define the neural network architecture for predicting the x and y values
class PointModel(nn.Module):
    def __init__(self):
        super(PointModel, self).__init__()
        self.fc1 = nn.Linear(4, 16)
        self.fc2 = nn.Linear(16, 2)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# set up the training data loader using the custom dataset
train_data = {'x': '1;2;3;4', 'y': '2;5;9;10', 'type': 'scatter'}
train_dataset = GraphDataset(train_data)
train_loader = DataLoader(train_dataset, batch_size=1)

# set up the neural network and training parameters
epochs = 100
lrate = 0.1
cost_fn = nn.MSELoss()
net = PointModel()
optimizer = torch.optim.Adam(net.parameters(), lr=lrate)


In [ ]:
for i, data in enumerate(train_loader, 0):
    inputs, = data
    optimizer.zero_grad()
    outputs = net(inputs)
    
    # convert the output values back to their original scale
    x_pred, y_pred = outputs[0].detach().numpy()
    x_pred = (x_pred * (max(train_dataset.x.flatten()) - min(train_dataset.x.flatten()))) + min(train_dataset.x.flatten())
    y_pred = (y_pred * (max(train_dataset.y.flatten()) - min(train_dataset.y.flatten()))) + min(train_dataset.y.flatten())
    
    # calculate the loss and update the neural network parameters
    x, y, graph_type = inputs.detach().numpy()
    x_true, y_true = x[:4], y[:4]
    loss = cost_fn(outputs, torch.tensor([x_true, y_true]).T)
    loss.backward()
    optimizer.step()
    cur_cost += loss.item()
    
track_cost[epoch] = cur_cost
print(f"Epoch: {epoch} Cost: {cur_cost}")


In [ ]:
#TODO
#EXTRACT INFO ABOUT SCATTER PLOTS
#EXTEND A PRETRAINED CAPTIONING MODEL TO CAPTION THOSE PLOTS
#DO ONE MODEL FOR EACH TYPE OF GRAPH